# Movie Sentiment Analysis
https://www.kaggle.com/c/word2vec-nlp-tutorial/

 拿到数据首先读入拿到数据

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import pandas as pd


train = pd.read_csv('labeledTrainData.tsv', delimiter="\t")
test = pd.read_csv('testData.tsv', delimiter="\t")
train.head()                

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [2]:
print (train.shape)
print (test.shape)


(25000, 3)
(25000, 2)


In [3]:

import re  #正则表达式

def review_preprocessing(review):
    #只保留英文单词
    review_text = re.sub("[^a-zA-Z]"," ", review)
    
    #变成小写
    words = review_text.lower()
    
    return(words)

# 把训练集的文本和标注分开

y_train=train.sentiment
y_test=train.sentiment

reviews=train.review
X_train=[]

for review in reviews:
    words=review_preprocessing(review)
    X_train.append(words)
    
  
X_train=np.array(X_train)


reviews_test=test.review
X_test=[]

for review in reviews_test:
    words=review_preprocessing(review)
    X_test.append(words)
X_test=np.array(X_test)


In [4]:
#print(X_test[0],'------',y_test[0],'------',X_train[0],'------',y_train[0])    

In [5]:
from sklearn.model_selection import train_test_split


X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)



In [6]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=CountVectorizer()



X_train_occurrence=vectorizer.fit_transform(X_train)
X_valid_occurrence=vectorizer.transform(X_valid)
X_test_occurrence= vectorizer.transform(X_test)



In [7]:
# 使用tf-idf
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train)
X_valid_tfidf=tfidf.transform(X_valid)
X_test_tfidf=tfidf.transform(X_test)

features=tfidf.get_feature_names()
#print(features)

In [8]:
# 多项式朴素贝叶斯
from sklearn.naive_bayes import MultinomialNB 
classifier=MultinomialNB()

classifier.fit(X_train_occurrence,y_train)
prediction_valid=classifier.predict(X_valid_occurrence)
prediction=classifier.predict(X_test_occurrence)


In [9]:
tfidf_classifier=MultinomialNB()
tfidf_classifier.fit(X_train_tfidf,y_train)
tfidf_valid=tfidf_classifier.predict(X_valid_tfidf)
tfidf_prediction=tfidf_classifier.predict(X_test_tfidf)


In [10]:
from sklearn.metrics import classification_report,confusion_matrix
print (classification_report(y_valid, prediction_valid))
print (confusion_matrix(y_valid, prediction_valid))

              precision    recall  f1-score   support

           0       0.83      0.88      0.85      2481
           1       0.87      0.82      0.84      2519

    accuracy                           0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000

[[2175  306]
 [ 453 2066]]


In [11]:
print (classification_report(y_valid, tfidf_valid))
print (confusion_matrix(y_valid, tfidf_valid))


#ok. 看来TFIDF 比普通的NB 好一点点

              precision    recall  f1-score   support

           0       0.85      0.88      0.86      2481
           1       0.88      0.84      0.86      2519

    accuracy                           0.86      5000
   macro avg       0.86      0.86      0.86      5000
weighted avg       0.86      0.86      0.86      5000

[[2189  292]
 [ 399 2120]]


In [12]:
# 把结果保存到csv文件中，并进行提交: https://www.kaggle.com/c/word2vec-nlp-tutorial/leaderboard
df = pd.DataFrame({"id": test['id'],"sentiment": tfidf_prediction})
df.to_csv('submission.csv',index = False, header=True)